# LLM Feasibility


In [1]:
!pip install -q transformers datasets pandas

## Load and Preprocess Data

In [1]:
import pandas as pd
import kagglehub

# Download and load dataset
path = kagglehub.dataset_download("piyushjain16/amazon-product-data")

df=pd.read_csv(path+"/dataset/train.csv")
# Filter relevant columns
df = df[['TITLE', 'BULLET_POINTS', 'DESCRIPTION', 'PRODUCT_TYPE_ID', 'PRODUCT_LENGTH']].dropna()

# Sample for speed
df = df.sample(20, random_state=42).reset_index(drop=True)
df.head()


,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
0,"Plane Light System, Plastic + Metal Taxi Light...",[Full set of bright LED lights for your model ...,Features:&nbsp;<br> Full set of bright LED lig...,1149,590.551180
1,"DECOR Kafe Home Decor Sunflower Wall Sticker, ...","[Size Small - 40cms X 76cms & Color Brown,Cust...",Welcome To The Foremost Place On The Web To Fi...,6030,1600.000000
2,Vbuyz Women's Rayon Foil Print Stitched Straig...,[Kurti (top) Fabric : Rayon || Kurti (top) Sty...,Vbuyz women's green color rayon straight kurti...,2916,1220.472440
3,Mitsui Shop on Suruga Street in Edo by Katsush...,"[High Resolution Artwork: Over 100,000 designs...",<p></p><br><p>Lost Cabin Art & Decor wall deco...,6548,2200.000000
4,"Brass Glass ( 1 pcs ), 250ml (Glass with Doubl...","[Specification:-Set fo 1, Material: Brass, Vol...","Specification:-Set for 1, Material: Brass, Vol...",1416,275.590551


In [2]:
df.info()
print('---'*20)
print(df.describe())
print('---'*20)
print('Dataset shape:',df.shape)
print('---'*20)
print('Missing Values:')
df.isnull().sum().sort_values(ascending=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   TITLE            20 non-null     object 
 1   BULLET_POINTS    20 non-null     object 
 2   DESCRIPTION      20 non-null     object 
 3   PRODUCT_TYPE_ID  20 non-null     int64  
 4   PRODUCT_LENGTH   20 non-null     float64
dtypes: float64(1), int64(1), object(3)
memory usage: 932.0+ bytes
------------------------------------------------------------
       PRODUCT_TYPE_ID  PRODUCT_LENGTH
count         20.00000       20.000000
mean        6767.40000     1199.455111
std         4310.38708     1657.000860
min         1149.00000       66.929000
25%         3255.00000      305.118110
50%         6513.50000      893.700787
75%        12058.25000     1265.354330
max        13104.00000     7800.000000
------------------------------------------------------------
Dataset shape: (20, 5)
-------------

,0
TITLE,0
BULLET_POINTS,0
DESCRIPTION,0
PRODUCT_TYPE_ID,0
PRODUCT_LENGTH,0


## Generate Descriptions with a Transformer Model

In [3]:
from transformers import pipeline

# Load model (FLAN-T5)
generator = pipeline("text2text-generation", model="google/flan-t5-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


In [4]:
def make_prompt(row):
    return f"""Generate a product description using the following metadata:

Title: {row['TITLE']}
Bullet Points: {row['BULLET_POINTS']}
Product Type ID: {row['PRODUCT_TYPE_ID']}
Length: {row['PRODUCT_LENGTH']}"""

# Apply model
generated = []
for _, row in df.iterrows():
    prompt = make_prompt(row)
    output = generator(prompt, max_length=128)[0]['generated_text']
    generated.append(output)

df['GENERATED_DESCRIPTION'] = generated
df[['TITLE', 'DESCRIPTION', 'GENERATED_DESCRIPTION']].head()


Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


,TITLE,DESCRIPTION,GENERATED_DESCRIPTION
0,"Plane Light System, Plastic + Metal Taxi Light...",Features:&nbsp;<br> Full set of bright LED lig...,"Plane Light System, Plastic + Metal Taxi Light..."
1,"DECOR Kafe Home Decor Sunflower Wall Sticker, ...",Welcome To The Foremost Place On The Web To Fi...,"DECOR Kafe Home Decor Sunflower Wall Sticker, ..."
2,Vbuyz Women's Rayon Foil Print Stitched Straig...,Vbuyz women's green color rayon straight kurti...,Vbuyz Women's Rayon Foil Print Stitched Straig...
3,Mitsui Shop on Suruga Street in Edo by Katsush...,<p></p><br><p>Lost Cabin Art & Decor wall deco...,Framed Canvas Wall Art Decor | Fine Artwork Pa...
4,"Brass Glass ( 1 pcs ), 250ml (Glass with Doubl...","Specification:-Set for 1, Material: Brass, Vol...","[Specification:-Set fo 1, Material: Brass, Vol..."


In [9]:
df=pd.read_csv(path+"/dataset/train.csv")
df = df.dropna(subset=['TITLE', 'DESCRIPTION'])
df = df.head(100)

inputs = df['TITLE'].astype(str).tolist()
refs = df['DESCRIPTION'].astype(str).tolist()

In [10]:
from transformers import pipeline

generator = pipeline("text2text-generation", model="google/flan-t5-small")

preds = []
for title in inputs:
    prompt = f"Generate a product description for this product: {title}"
    result = generator(prompt, max_new_tokens=60)
    preds.append(result[0]['generated_text'])

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [5]:
# vision for product descriptions

for i in range(5):
    print(f"\n🔷 TITLE: {df.loc[i, 'TITLE']}")
    print(f"📘 Original Description:\n{df.loc[i, 'DESCRIPTION']}")
    print(f"🤖 Generated:\n{df.loc[i, 'GENERATED_DESCRIPTION']}")


🔷 TITLE: Plane Light System, Plastic + Metal Taxi Lights Airplane LED Light, for Model Plane
📘 Original Description:
Features:&nbsp;<br> Full set of bright LED lights for your model plane or glider. <br> Use the power supplied by the receiver and don't require its own battery. <br> The circuitry has been specially designed so that the landing lights and taxi lights can be switched on and off directly. <br> Either use a Y-harness and link it to another channel such as the gear or assign a separate channel to it. <br> It includes red beacon light, white taxi light, white strobe light, white landing light, green navigation light, red navigation light. <br> <br>Specification:&nbsp;<br>Material: Plastic + Metal<br>Color: Shown As Pictures<br>Weight: 111g<br>Type: RC Part &amp; Accessory<br>Control Board Size: Approx. 54 * 35 * 13mm / 2.1 * 1.4 * 0.5inch<br> Cable Size: Approx. OD 0.8mm*0.8m <br> LED Quantity: 14pcs <br>Lighting System Specification:<br>2 x Red Beacon Lights (#11, #12), 600

## Evaluate Generated Descriptions

In [25]:
# Silent installs
import sys
import subprocess

def install(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "-q"])

install("textstat")
install("evaluate")
install("bert_score")
install("transformers")

from transformers import pipeline
import evaluate
from bert_score import score as bert_score
import textstat

# Load your dataset (update path as needed)
df=pd.read_csv(path+"/dataset/train.csv")
df = df.dropna(subset=['TITLE', 'DESCRIPTION'])
df = df.head(100)  # small sample for testing

inputs = df['TITLE'].astype(str).tolist()
refs = df['DESCRIPTION'].astype(str).tolist()

# Generate descriptions with a T5 model
generator = pipeline("text2text-generation", model="google/flan-t5-small")

print("Generating descriptions...")
preds = []
for title in inputs:
    prompt = f"Generate a product description for this product: {title}"
    result = generator(prompt, max_new_tokens=60)
    preds.append(result[0]['generated_text'])

# Compute BLEU and ROUGE
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

bleu_result = bleu.compute(predictions=preds, references=refs)
rouge_result = rouge.compute(predictions=preds, references=refs)

# Compute BERTScore
P, R, F1 = bert_score(preds, refs, lang="en")

# Compute average Flesch Reading Ease using textstat
def avg_flesch(texts):
    scores = [textstat.flesch_reading_ease(t) for t in texts if len(t.split()) > 2]
    return sum(scores) / len(scores) if scores else 0

orig_readability = avg_flesch(refs)
gen_readability = avg_flesch(preds)

print("\n=== Evaluation Results ===")
print(f"BLEU Score: {bleu_result['bleu']:.4f}")
print(f"ROUGE-1: {rouge_result['rouge1']:.4f}")
print(f"ROUGE-2: {rouge_result['rouge2']:.4f}")
print(f"ROUGE-L: {rouge_result['rougeL']:.4f}")
print(f"BERTScore Precision: {P.mean().item():.4f}")
print(f"BERTScore Recall:    {R.mean().item():.4f}")
print(f"BERTScore F1:        {F1.mean().item():.4f}")
print(f"Original Readability (Flesch): {orig_readability:.2f}")
print(f"Generated Readability (Flesch): {gen_readability:.2f}")


Device set to use cpu


Generating descriptions...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



=== Evaluation Results ===
BLEU Score: 0.0003
ROUGE-1: 0.2329
ROUGE-2: 0.1416
ROUGE-L: 0.2040
BERTScore Precision: 0.8500
BERTScore Recall:    0.8086
BERTScore F1:        0.8278
Original Readability (Flesch): 50.50
Generated Readability (Flesch): 57.04


In [22]:
original_lengths = [len(t.split()) for t in refs]
generated_lengths = [len(t.split()) for t in preds]

print("Avg Original Length:", np.mean(original_lengths))
print("Avg Generated Length:", np.mean(generated_lengths))

Avg Original Length: 90.05
Avg Generated Length: 16.8


Summary:
* The ROUGE scores are relatively low, suggesting limited direct overlap in the generated descriptions compared to the originals.
* The BLEU score of 0.0003 is very low, indicating minimal direct word overlap and structure similarity with the original descriptions.
* The BERTScore F1 of 0.828 suggests a relatively high  similarity between the
generated and original descriptions, even if the exact wording is different. This indicates the model is capturing the meaning well.
* The generated descriptions have a slightly higher average Flesch score, suggesting they are slightly easier to read than the original descriptions.


Although the generated descriptions don't have high direct word overlap with the originals (low ROUGE and BLEU), they appear to capture the semantic meaning reasonably well (high BERTScore) and are slightly easier to read.